In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
import re

In [2]:
def convert_to_number(text):
    """Chuyển đổi số trong định dạng 'x tỷ y triệu' thành dạng số"""
    text = text.lower().replace(' ', '')
    billion_index = text.find('tỷ')
    million_index = text.find('triệu')
    if billion_index == -1:
        billion = 0
    else:
        billion = int(text[:billion_index]) * 1000000000
    if million_index == -1:
        million = 0
    else:
        million = int(text[billion_index+2:million_index]) * 1000000
    number = billion + million
    return number

In [3]:
data = pd.DataFrame(columns=['Tên_xe', 'hãng_xe', 'link_bài_viết', 'loại_xe', 'nguồn_gốc', 'số_km_đã_lái', 'màu', 'số_ghế', 'dung_tích_xi_lanh', 'loại_nguyên_liệu', 'số_sàn_hay_tự_động', 'dẫn_động', 'giá', 'năm_sản_xuất'])
data

,Tên_xe,hãng_xe,link_bài_viết,loại_xe,nguồn_gốc,số_km_đã_lái,màu,số_ghế,dung_tích_xi_lanh,loại_nguyên_liệu,số_sàn_hay_tự_động,dẫn_động,giá,năm_sản_xuất


In [4]:
type_nhien_lieu = {'Dầu': 'diesel', 'Hybrid':'Hybrid', 'Điện': 'Điện', 'Xăng':'Xăng'}

In [5]:
data

,Tên_xe,hãng_xe,link_bài_viết,loại_xe,nguồn_gốc,số_km_đã_lái,màu,số_ghế,dung_tích_xi_lanh,loại_nguyên_liệu,số_sàn_hay_tự_động,dẫn_động,giá,năm_sản_xuất


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Khởi tạo trình duyệt
driver = webdriver.Chrome()

In [7]:
# print(f'https://bonbanh.com/oto/page,{i}')

In [8]:
id = 0
data = pd.DataFrame(columns=['Tên_xe', 'hãng_xe', 'link_bài_viết', 'loại_xe', 'nguồn_gốc', 'số_km_đã_lái', 'màu', 'số_ghế', 'dung_tích_xi_lanh', 'loại_nguyên_liệu', 'số_sàn_hay_tự_động', 'dẫn_động', 'giá', 'năm_sản_xuất'])
data

,Tên_xe,hãng_xe,link_bài_viết,loại_xe,nguồn_gốc,số_km_đã_lái,màu,số_ghế,dung_tích_xi_lanh,loại_nguyên_liệu,số_sàn_hay_tự_động,dẫn_động,giá,năm_sản_xuất


In [7]:

def process():
    id = 0
    for i in range(1, 2008):
        if id == 1000: return
        list_car = None
        html_text = requests.get(f'https://bonbanh.com/oto/page,{i}').text
        soup = BeautifulSoup(html_text, 'lxml')
        list_cars = soup.find('div', class_="gray-box")
        if list_cars == None:
            print(f'LOI TRANG: https://bonbanh.com/oto/page,{i}')
            continue
        else: 
            list_car = list_cars.find_all('a')
            
        print(f'https://bonbanh.com/oto/page,{i}')
        
        time.sleep(0.5)
        for car in list_car:
    
            if 'javascript' in car['href']: continue
            soup_sub = requests.get(f"https://bonbanh.com/{car['href']}").text
            time.sleep(0.5)
            soup_sub = BeautifulSoup(soup_sub, 'lxml')
            try:
                ten_xe, gia = soup_sub.find('h1').text.strip().replace('\t', ' ').split(' - ')
            except:
                print(soup_sub.find('h1').text.strip().split(' - '))
                # return
            ten_xe = " ".join(ten_xe.split())
            gia = convert_to_number(gia)
            nam = int(ten_xe.split(' ')[-1].strip())
            ### hang xe
            hang_xe = soup_sub.find_all('span', {'itemprop':"name"})[3].text
            info_car =soup_sub.find_all('span', class_ = 'inp')
            link_pape = car['href']
            
            loai_xe = info_car[2].text
            
            nguon_goc = info_car[0].text
            
            km = info_car[3].text
            
            mau = info_car[4].text
            
            so_ghe = info_car[7].text
            
            ### Nhien lieu
            dong_co = info_car[8].text
            nhien_lieu = None
            
            for nl in type_nhien_lieu.keys():
                if nl in dong_co:
                    nhien_lieu = type_nhien_lieu[nl]
                    break
            hop_so = info_car[10].text
            dan_dong = info_car[11].text
            driver.get(f"https://bonbanh.com/{car['href']}")
            login_button = driver.find_element(By.XPATH,"//*[@id='undefinednav4']")
            login_button.click()
            dung_tich_xi_lanh = driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/div[5]/div/div[4]/div[1]/div[10]/div[2]/span").text
            data.loc[id] = [ten_xe, hang_xe, link_pape, loai_xe, nguon_goc, km, mau, so_ghe, dung_tich_xi_lanh, nhien_lieu, hop_so, dan_dong, gia, nam]
            id+=1

In [8]:
process()

https://bonbanh.com/oto/page,1


KeyboardInterrupt: 

In [ ]:
data['link_bài_viết'] = data['link_bài_viết'].apply(lambda x: f'https://bonbanh.com/{x}')

In [ ]:
data.to_excel('data_ex.xlsx', index=False)

In [ ]:
data

,Tên_xe,hãng_xe,link_bài_viết,loại_xe,nguồn_gốc,số_km_đã_lái,màu,số_ghế,dung_tích_xi_lanh,loại_nguyên_liệu,số_sàn_hay_tự_động,dẫn_động,giá,năm_sản_xuất
0,Xe VinFast Lux A 2.0 Base 2021,VinFast,https://bonbanh.com/xe-vinfast-lux_a_2.0-base-...,Sedan,Lắp ráp trong nước,"41,000 Km",Đen,5 chỗ,,Xăng,Số tự động,RFD - Dẫn động cầu sau,39000000,2021
1,Xe VinFast Lux A 2.0 Base 2022,VinFast,https://bonbanh.com/xe-vinfast-lux_a_2.0-base-...,Sedan,Lắp ráp trong nước,"6,600 Km",Đỏ,5 chỗ,,Xăng,Số tự động,RFD - Dẫn động cầu sau,66000000,2022
2,Xe BMW X7 SUV 2020,BMW,https://bonbanh.com/xe-bmw-x7-suv-2020-4935736,SUV,Nhập khẩu,"40,000 Km",-,6 chỗ,,Xăng,Số tự động,4WD - Dẫn động 4 bánh,4690000000,2020
3,Xe Lexus RX 350L 2021,Lexus,https://bonbanh.com/xe-lexus-rx-350l-2021-4874881,SUV,Nhập khẩu,"16,000 Km",Trắng,7 chỗ,3.5 L,Xăng,Số tự động,AWD - 4 bánh toàn thời gian,3680000000,2021
4,Xe VinFast Lux SA 2.0 Plus 2.0 AT 2021,VinFast,https://bonbanh.com/xe-vinfast-lux_sa_2.0-plus...,SUV,Lắp ráp trong nước,"16,000 Km",Đỏ,7 chỗ,2.0 L,Xăng,Số tự động,RFD - Dẫn động cầu sau,86000000,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Xe Mazda 3 1.5 AT 2017,Mazda,https://bonbanh.com/xe-mazda-3-1.5-at-2017-482...,Sedan,Lắp ráp trong nước,"60,000 Km",Xanh,5 chỗ,1.5 L,Xăng,Số tự động,FWD - Dẫn động cầu trước,10000000,2017
996,Xe Mercedes Benz E class E250 2011,Mercedes Benz,https://bonbanh.com/xe-mercedes_benz-e_class-e...,Sedan,Lắp ráp trong nước,"110,000 Km",Đen,5 chỗ,1.8 L,Xăng,Số tự động,RFD - Dẫn động cầu sau,19000000,2011
997,Xe Toyota Wigo 2023,Toyota,https://bonbanh.com/xe-toyota-wigo--2023-4852855,Hatchback,Nhập khẩu,0 Km,Đen,5 chỗ,,Xăng,Số tự động,FWD - Dẫn động cầu trước,79000000,2023
998,Xe Honda CRV L 2023,Honda,https://bonbanh.com/xe-honda-crv-l-2023-3626733,SUV,Lắp ráp trong nước,0 Km,Xanh,7 chỗ,1.5 L,Xăng,Số tự động,FWD - Dẫn động cầu trước,1118000000,2023


In [ ]:
# for i in range(len(data)):
#     if data['loại_nguyên_liệu'][i] == None:
#         # print(data['link_bài_viết'][i])
#         soup_sub = requests.get(data['link_bài_viết'][i]).text
#         # print(soup_sub)
#         soup_sub = BeautifulSoup(soup_sub, 'lxml')
#         info_car =soup_sub.find_all('span', class_ = 'inp')
#         dong_co = info_car[8].text
#         nhien_lieu = None
#         for nl in type_nhien_lieu.keys():
#             if nl in dong_co:
#                 data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]
#                 print(data['loại_nguyên_liệu'][i])
#                 break

C:\Users\nguye\AppData\Local\Temp\ipykernel_15504\3460446175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]


Xăng


C:\Users\nguye\AppData\Local\Temp\ipykernel_15504\3460446175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]


Xăng


C:\Users\nguye\AppData\Local\Temp\ipykernel_15504\3460446175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]


...

C:\Users\nguye\AppData\Local\Temp\ipykernel_15504\3460446175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]


Xăng


C:\Users\nguye\AppData\Local\Temp\ipykernel_15504\3460446175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]


Xăng
diesel


C:\Users\nguye\AppData\Local\Temp\ipykernel_15504\3460446175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loại_nguyên_liệu'][i] = type_nhien_lieu[nl]


In [ ]:
data

,Tên_xe,hãng_xe,link_bài_viết,loại_xe,nguồn_gốc,số_km_đã_lái,màu,số_ghế,dung_tích_xi_lanh,loại_nguyên_liệu,số_sàn_hay_tự_động,dẫn_động,giá,năm_sản_xuất
0,Xe VinFast Lux A 2.0 Base 2021,VinFast,https://bonbanh.com/xe-vinfast-lux_a_2.0-base-...,Sedan,Lắp ráp trong nước,"41,000 Km",Đen,5 chỗ,,Xăng,Số tự động,RFD - Dẫn động cầu sau,39000000,2021
1,Xe VinFast Lux A 2.0 Base 2022,VinFast,https://bonbanh.com/xe-vinfast-lux_a_2.0-base-...,Sedan,Lắp ráp trong nước,"6,600 Km",Đỏ,5 chỗ,,Xăng,Số tự động,RFD - Dẫn động cầu sau,66000000,2022
2,Xe BMW X7 SUV 2020,BMW,https://bonbanh.com/xe-bmw-x7-suv-2020-4935736,SUV,Nhập khẩu,"40,000 Km",-,6 chỗ,,Xăng,Số tự động,4WD - Dẫn động 4 bánh,4690000000,2020
3,Xe Lexus RX 350L 2021,Lexus,https://bonbanh.com/xe-lexus-rx-350l-2021-4874881,SUV,Nhập khẩu,"16,000 Km",Trắng,7 chỗ,3.5 L,Xăng,Số tự động,AWD - 4 bánh toàn thời gian,3680000000,2021
4,Xe VinFast Lux SA 2.0 Plus 2.0 AT 2021,VinFast,https://bonbanh.com/xe-vinfast-lux_sa_2.0-plus...,SUV,Lắp ráp trong nước,"16,000 Km",Đỏ,7 chỗ,2.0 L,Xăng,Số tự động,RFD - Dẫn động cầu sau,86000000,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Xe Mazda 3 1.5 AT 2017,Mazda,https://bonbanh.com/xe-mazda-3-1.5-at-2017-482...,Sedan,Lắp ráp trong nước,"60,000 Km",Xanh,5 chỗ,1.5 L,Xăng,Số tự động,FWD - Dẫn động cầu trước,10000000,2017
996,Xe Mercedes Benz E class E250 2011,Mercedes Benz,https://bonbanh.com/xe-mercedes_benz-e_class-e...,Sedan,Lắp ráp trong nước,"110,000 Km",Đen,5 chỗ,1.8 L,Xăng,Số tự động,RFD - Dẫn động cầu sau,19000000,2011
997,Xe Toyota Wigo 2023,Toyota,https://bonbanh.com/xe-toyota-wigo--2023-4852855,Hatchback,Nhập khẩu,0 Km,Đen,5 chỗ,,Xăng,Số tự động,FWD - Dẫn động cầu trước,79000000,2023
998,Xe Honda CRV L 2023,Honda,https://bonbanh.com/xe-honda-crv-l-2023-3626733,SUV,Lắp ráp trong nước,0 Km,Xanh,7 chỗ,1.5 L,Xăng,Số tự động,FWD - Dẫn động cầu trước,1118000000,2023
